# **환경 설정**

In [ ]:
! pip install langchain_community tiktoken langchain-openai chromadb langchain langgraph tavily-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")